## Zero-dependency “agent”

**Goal: simulate the agent loop with 3 tools**

In [1]:
# "search" over a tiny in-memory corpus ---
DOCS = {
    "germany_gdp": "Germany GDP in 2023 was about 4.4 trillion USD (nominal).",
    "france_gdp":  "France GDP in 2023 was about 3.1 trillion USD (nominal).",
    "heidelberg":  "Heidelberg is a university town on the Neckar River in southwest Germany."
}

def search_tool(query: str) -> str:
    q = query.lower()
    # naive keyword lookup
    if "germany" in q and "gdp" in q: return DOCS["germany_gdp"]
    if "france"  in q and "gdp" in q: return DOCS["france_gdp"]
    if "heidelberg" in q:             return DOCS["heidelberg"]
    return "No result found."

def calculator_tool(expr: str) -> str:
    # very small safe parser: only digits and +-*./()
    import re
    if not re.fullmatch(r"[0-9\.\+\-\*\/\(\) ]+", expr):
        return "Calculator error: invalid expression."
    try:
        return str(eval(expr))
    except Exception as e:
        return f"Calculator error: {e}"

def summarizer_tool(text: str, max_words: int = 18) -> str:
    words = text.split()
    return " ".join(words[:max_words]) + ("…" if len(words) > max_words else "")


## Minimal agent loop (Reason → Act → Observe → Repeat)

In [2]:
MEMORY = []

def remember(event: str):
    MEMORY.append(event)

def agent(goal: str):
    print(f" Goal: {goal}")
    remember(f"GOAL::{goal}")

    if "gdp" in goal.lower() and "germany" in goal.lower() and "france" in goal.lower():
        # plan
        remember("PLAN::search GDPs, compute difference, summarize")
        # act 1
        de = search_tool("Germany GDP 2023")
        fr = search_tool("France GDP 2023")
        remember(f"OBS::DE={de} | FR={fr}")

        # extract numbers (toy)
        import re
        de_num = float(re.findall(r"([0-9\.]+) trillion", de)[0])
        fr_num = float(re.findall(r"([0-9\.]+) trillion", fr)[0])

        # act 2: calculate difference
        diff = calculator_tool(f"{de_num} - {fr_num}")
        remember(f"OBS::DIFF={diff}")

        # act 3: summarize
        out = summarizer_tool(
            f"Germany GDP ~ {de_num}T; France ~ {fr_num}T; delta ~ {diff}T USD."
        )
        remember(f"ANSWER::{out}")
        return out

    # fallback behavior
    text = search_tool(goal)
    out = summarizer_tool(text)
    remember(f"ANSWER::{out}")
    return out

print(agent("Compare GDP of Germany and France for 2023 and give me the difference"))
print(" Memory trace:", MEMORY)


 Goal: Compare GDP of Germany and France for 2023 and give me the difference
Germany GDP ~ 4.4T; France ~ 3.1T; delta ~ 1.3000000000000003T USD.
 Memory trace: ['GOAL::Compare GDP of Germany and France for 2023 and give me the difference', 'PLAN::search GDPs, compute difference, summarize', 'OBS::DE=Germany GDP in 2023 was about 4.4 trillion USD (nominal). | FR=France GDP in 2023 was about 3.1 trillion USD (nominal).', 'OBS::DIFF=1.3000000000000003', 'ANSWER::Germany GDP ~ 4.4T; France ~ 3.1T; delta ~ 1.3000000000000003T USD.']
